## Instructions
To move forward with the project, you need to create a collection of songs with their audio features - as large as possible! <br>
<br>
These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards. <br>
<br>
The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.<br>
<br>
An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

### Import libraries

In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from getpass import getpass
from random import sample

### Initialize SpotiPy with user credentials

In [2]:
client_id = getpass("Insert client ID: ")
client_secret = getpass("Insert client secret: ")

Insert client ID: ········
Insert client secret: ········


In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id, client_secret))

### Load playlist

In [4]:
# Check the name of the playlist

playlist = sp.playlist("37i9dQZF1DWUbj5dNZ89An")

In [5]:
playlist["name"]

'Canta en el coche'

### Obtain the name and id of the tracks

In [6]:
playlist = sp.user_playlist_tracks("spotify", "37i9dQZF1DWUbj5dNZ89An")
playlist.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [7]:
playlist["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [8]:
playlist["items"][0]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [9]:
playlist["items"][0]["track"]["name"]

'La Bebe - Remix'

In [10]:
playlist["items"][0]["track"]["id"]

'2UW7JaomAMuX9pZrjVpHAU'

In [11]:
artists = []
song_artists = []

for artist in playlist["items"][99]["track"]["artists"]:
    song_artists.append(artist["name"])

joined_song_artists = ", ".join(song_artists)
artists.append(joined_song_artists)
artists

['Aitana, Emilia, Ptazeta']

In [12]:
# Function to obtain all the tracks titles, ids and artists from a playlist created by Spotify

def get_track_id_title(playlist_id):
    
    titles = []
    ids = []
    artists = []

    playlist = sp.user_playlist_tracks("spotify", playlist_id)
    for i in range(0, len(playlist["items"])):
        song_artists = []
        title = playlist["items"][i]["track"]["name"]
        song_id = playlist["items"][i]["track"]["id"]
        for artist in playlist["items"][i]["track"]["artists"]:
            song_artists.append(artist["name"])
        titles.append(title)
        ids.append(song_id)
        joined_song_artists = ", ".join(song_artists)
        artists.append(joined_song_artists)
        
    return titles, ids, artists

In [13]:
titles_canta_coche, ids_canta_coche, artists_canta_coche = get_track_id_title("37i9dQZF1DWUbj5dNZ89An")

In [14]:
sample(titles_canta_coche, 10)

['Todo Contigo',
 'La Fiesta',
 'Until I Found You',
 'Quieres',
 'Manos Frías (con Reik, Beéle)',
 '23',
 'PLAYA DEL INGLÉS',
 'Ojitos Lindos',
 'Y, ¿Si Fuera Ella? - + Es +',
 'El Fin del Mundo']

In [15]:
sample(ids_canta_coche, 10)

['2xVcCDRgG3TrH69TatsUxp',
 '5CM4UuQ9Gnd6K2YyKGPMoK',
 '0IqCoZ168iRc9LqfrYgpZy',
 '3k3NWokhRRkEPhCzPmV8TW',
 '3USxtqRwSYz57Ewm6wWRMp',
 '3iHzKA9HlXf5wsGdsrsnSA',
 '7bfFmlLJ1LKr2E3AUsvq5P',
 '6OzUIp8KjuwxJnCWkXp1uL',
 '6G12ZafqofSq7YtrMqUm76',
 '3XjN0mvFkQp1FMVM9Ro5hm']

In [16]:
sample(artists_canta_coche, 10)

['Wisin & Yandel, ROSALÍA',
 'Lil Durk, J. Cole',
 'Rels B',
 'KAROL G',
 'India Martinez, Melendi',
 'zzoilo, Aitana',
 'Quevedo, Myke Towers',
 'KAROL G',
 'Melendi, Ha*Ash',
 'Marta Santos']

### Obtain audio features

In [17]:
# sp.audio_features(ids_canta_coche)

### Create dataframe

In [18]:
canta_coche = pd.json_normalize(sp.audio_features(ids_canta_coche))
canta_coche["title"] = titles_canta_coche 
canta_coche["artist"] = artists_canta_coche 
canta_coche.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.812,0.479,2,-5.678,0,0.3330,0.2130,0.000001,0.0756,0.559,169.922,audio_features,2UW7JaomAMuX9pZrjVpHAU,spotify:track:2UW7JaomAMuX9pZrjVpHAU,https://api.spotify.com/v1/tracks/2UW7JaomAMuX...,https://api.spotify.com/v1/audio-analysis/2UW7...,234353,4,La Bebe - Remix,"Yng Lvcas, Peso Pluma"
1,0.652,0.800,9,-4.019,0,0.0614,0.1430,0.629000,0.1120,0.234,143.978,audio_features,7ro0hRteUMfnOioTFI5TG1,spotify:track:7ro0hRteUMfnOioTFI5TG1,https://api.spotify.com/v1/tracks/7ro0hRteUMfn...,https://api.spotify.com/v1/audio-analysis/7ro0...,231704,4,WHERE SHE GOES,Bad Bunny
2,0.748,0.761,2,-6.621,1,0.1020,0.1220,0.000007,0.0984,0.362,116.041,audio_features,5TMLavqGRKNxgzMBrVpC9R,spotify:track:5TMLavqGRKNxgzMBrVpC9R,https://api.spotify.com/v1/tracks/5TMLavqGRKNx...,https://api.spotify.com/v1/audio-analysis/5TML...,179940,4,Mon Amour - Remix,"zzoilo, Aitana"
3,0.748,0.856,11,-3.517,0,0.0348,0.0824,0.000000,0.1420,0.927,123.031,audio_features,0Be7sopyKMv8Y8npsUkax2,spotify:track:0Be7sopyKMv8Y8npsUkax2,https://api.spotify.com/v1/tracks/0Be7sopyKMv8...,https://api.spotify.com/v1/audio-analysis/0Be7...,189320,4,Tacones Rojos,Sebastian Yatra
4,0.783,0.592,11,-8.332,0,0.0331,0.4350,0.000004,0.3470,0.726,120.018,audio_features,7FbrGaHYVDmfr7KoLIZnQ7,spotify:track:7FbrGaHYVDmfr7KoLIZnQ7,https://api.spotify.com/v1/tracks/7FbrGaHYVDmf...,https://api.spotify.com/v1/audio-analysis/7Fbr...,174253,4,Cupid - Twin Ver.,FIFTY FIFTY


### Repeat with more playlists

In [19]:
# Function to obtain audio features and create dataframe

def df_audio_features(playlist_id):
    titles, ids, artists = get_track_id_title(playlist_id)
    audio_features = sp.audio_features(ids)
    df = pd.json_normalize(audio_features)
    df["title"] = titles
    df["artist"] = artists
    return df

In [20]:
df_list = [canta_coche]

In [21]:
country_hits = df_audio_features("37i9dQZF1DWZBCPUIUs2iR")
df_list.append(country_hits)
country_hits.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.554,0.783,6,-5.852,1,0.0457,0.4160,0.000002,0.6310,0.813,105.390,audio_features,4w3tQBXhn5345eUXDGBWZG,spotify:track:4w3tQBXhn5345eUXDGBWZG,https://api.spotify.com/v1/tracks/4w3tQBXhn534...,https://api.spotify.com/v1/audio-analysis/4w3t...,162467,4,9 to 5,Dolly Parton
1,0.614,0.680,6,-4.580,1,0.0289,0.3690,0.000002,0.1150,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4,Whiskey Glasses,Morgan Wallen
2,0.708,0.524,5,-12.644,1,0.0352,0.0551,0.000003,0.0371,0.690,111.482,audio_features,6iy4PoAuZBMvtrlDX4VxC7,spotify:track:6iy4PoAuZBMvtrlDX4VxC7,https://api.spotify.com/v1/tracks/6iy4PoAuZBMv...,https://api.spotify.com/v1/audio-analysis/6iy4...,200667,4,Check Yes Or No,George Strait
3,0.657,0.639,10,-7.527,1,0.0412,0.5810,0.000000,0.2780,0.674,124.882,audio_features,6sxptembJVty4sNtcPMAVz,spotify:track:6sxptembJVty4sNtcPMAVz,https://api.spotify.com/v1/tracks/6sxptembJVty...,https://api.spotify.com/v1/audio-analysis/6sxp...,234493,4,Man! I Feel Like A Woman!,Shania Twain
4,0.671,0.501,4,-13.119,1,0.0594,0.3420,0.000000,0.1940,0.860,87.040,audio_features,5KqldkCunQ2rWxruMEtGh0,spotify:track:5KqldkCunQ2rWxruMEtGh0,https://api.spotify.com/v1/tracks/5KqldkCunQ2r...,https://api.spotify.com/v1/audio-analysis/5Kql...,214880,4,The Gambler,Kenny Rogers


In [22]:
iconic_soundtracks = df_audio_features("37i9dQZF1DX1tz6EDao8it")
df_list.append(iconic_soundtracks)
iconic_soundtracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.465,0.6870,10,-10.964,1,0.0495,0.0206,0.929,0.1210,0.1300,70.507,audio_features,08QaHlMPWuO5PUxjl61bXn,spotify:track:08QaHlMPWuO5PUxjl61bXn,https://api.spotify.com/v1/tracks/08QaHlMPWuO5...,https://api.spotify.com/v1/audio-analysis/08Qa...,90427,4,He's a Pirate,Klaus Badelt
1,0.286,0.0968,7,-16.806,1,0.0323,0.1780,0.696,0.0861,0.0395,62.881,audio_features,6ZFbXIJkuI1dVNWvzJzown,spotify:track:6ZFbXIJkuI1dVNWvzJzown,https://api.spotify.com/v1/tracks/6ZFbXIJkuI1d...,https://api.spotify.com/v1/audio-analysis/6ZFb...,275556,4,Time,Hans Zimmer
2,0.144,0.1320,10,-19.115,1,0.0333,0.9460,0.779,0.1150,0.0627,102.283,audio_features,2TZbQZXOuR8osP2AK8yYMN,spotify:track:2TZbQZXOuR8osP2AK8yYMN,https://api.spotify.com/v1/tracks/2TZbQZXOuR8o...,https://api.spotify.com/v1/audio-analysis/2TZb...,207427,3,Theme From Jurassic Park,John Williams
3,0.180,0.2260,9,-16.457,0,0.0420,0.9510,0.982,0.0998,0.0480,94.079,audio_features,6pWgRkpqVfxnj3WuIcJ7WP,spotify:track:6pWgRkpqVfxnj3WuIcJ7WP,https://api.spotify.com/v1/tracks/6pWgRkpqVfxn...,https://api.spotify.com/v1/audio-analysis/6pWg...,126960,4,Cornfield Chase,Hans Zimmer
4,0.483,0.1920,4,-15.378,0,0.0469,0.9910,0.908,0.1060,0.0542,100.848,audio_features,14rZjW3RioG7WesZhYESso,spotify:track:14rZjW3RioG7WesZhYESso,https://api.spotify.com/v1/tracks/14rZjW3RioG7...,https://api.spotify.com/v1/audio-analysis/14rZ...,140733,4,"Comptine d'un autre été, l'après-midi",Yann Tiersen


In [23]:
samba_classics = df_audio_features("37i9dQZF1DX2OwDeeyTh0V")
df_list.append(samba_classics)
samba_classics.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.674,0.419,7,-12.413,0,0.2680,0.747,0.000000,0.0797,0.870,150.131,audio_features,7fs6csqE3EltCrESv6cn0J,spotify:track:7fs6csqE3EltCrESv6cn0J,https://api.spotify.com/v1/tracks/7fs6csqE3Elt...,https://api.spotify.com/v1/audio-analysis/7fs6...,144760,4,Disritmia,Martinho Da Vila
1,0.596,0.451,5,-9.468,1,0.0404,0.640,0.000004,0.1580,0.486,148.045,audio_features,5Ikos3uiBfPYAf7EZfz8t3,spotify:track:5Ikos3uiBfPYAf7EZfz8t3,https://api.spotify.com/v1/tracks/5Ikos3uiBfPY...,https://api.spotify.com/v1/audio-analysis/5Iko...,228133,4,Sonho Meu,"Sambabook, Arlindo Cruz, Maria Bethânia, Delci..."
2,0.605,0.662,10,-6.993,1,0.0924,0.432,0.000000,0.1160,0.953,93.771,audio_features,0m3hiKjI2QGSKxvG732Ekq,spotify:track:0m3hiKjI2QGSKxvG732Ekq,https://api.spotify.com/v1/tracks/0m3hiKjI2QGS...,https://api.spotify.com/v1/audio-analysis/0m3h...,162360,4,Tiro Ao Álvaro,"Elis Regina, Adoniran Barbosa"
3,0.695,0.924,9,-4.178,1,0.0454,0.229,0.000000,0.2640,0.964,94.827,audio_features,0rImThhPOV8wtgeyltD6no,spotify:track:0rImThhPOV8wtgeyltD6no,https://api.spotify.com/v1/tracks/0rImThhPOV8w...,https://api.spotify.com/v1/audio-analysis/0rIm...,232800,4,Malandragem dá um Tempo,Bezerra Da Silva
4,0.705,0.337,11,-14.457,1,0.3430,0.284,0.000003,0.0747,0.600,96.556,audio_features,3C8ULtjiUmTtcpE7QJBZ7G,spotify:track:3C8ULtjiUmTtcpE7QJBZ7G,https://api.spotify.com/v1/tracks/3C8ULtjiUmTt...,https://api.spotify.com/v1/audio-analysis/3C8U...,205200,4,Timoneiro,Paulinho Da Viola


In [24]:
feelin_good = df_audio_features("37i9dQZF1DX9XIFQuFvzM4")
df_list.append(feelin_good)
feelin_good.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.663,0.600,7,-10.870,1,0.0320,0.430,0.000000,0.1840,0.800,129.991,audio_features,7tqhbajSfrz2F7E1Z75ASX,spotify:track:7tqhbajSfrz2F7E1Z75ASX,https://api.spotify.com/v1/tracks/7tqhbajSfrz2...,https://api.spotify.com/v1/audio-analysis/7tqh...,151667,4,Ain't No Mountain High Enough,"Marvin Gaye, Tammi Terrell"
1,0.527,0.415,4,-11.451,0,0.1220,0.457,0.000017,0.1170,0.515,78.169,audio_features,1k1Bqnv2R0uJXQN4u6LKYt,spotify:track:1k1Bqnv2R0uJXQN4u6LKYt,https://api.spotify.com/v1/tracks/1k1Bqnv2R0uJ...,https://api.spotify.com/v1/audio-analysis/1k1B...,125093,4,Ain't No Sunshine,Bill Withers
2,0.650,0.306,9,-9.443,1,0.0393,0.570,0.000007,0.0707,0.605,118.068,audio_features,3SdTKo2uVsxFblQjpScoHy,spotify:track:3SdTKo2uVsxFblQjpScoHy,https://api.spotify.com/v1/tracks/3SdTKo2uVsxF...,https://api.spotify.com/v1/audio-analysis/3SdT...,180056,4,Stand by Me,Ben E. King
3,0.572,0.418,0,-10.738,1,0.0349,0.635,0.000000,0.0961,0.694,104.566,audio_features,745H5CctFr12Mo7cqa1BMH,spotify:track:745H5CctFr12Mo7cqa1BMH,https://api.spotify.com/v1/tracks/745H5CctFr12...,https://api.spotify.com/v1/audio-analysis/745H...,165000,4,My Girl,The Temptations
4,0.769,0.367,2,-11.226,1,0.0312,0.684,0.000016,0.0810,0.535,103.621,audio_features,3zBhihYUHBmGd2bcQIobrF,spotify:track:3zBhihYUHBmGd2bcQIobrF,https://api.spotify.com/v1/tracks/3zBhihYUHBmG...,https://api.spotify.com/v1/audio-analysis/3zBh...,163756,4,(Sittin' On) the Dock of the Bay,Otis Redding


In [25]:
summer_jazz = df_audio_features("37i9dQZF1DWTKxc7ZObqeH")
df_list.append(summer_jazz)
summer_jazz.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.557,0.229,7,-15.943,1,0.0359,0.853,0.423,0.0984,0.0816,61.858,audio_features,1bB1538FOBYVUs2z06ssQ3,spotify:track:1bB1538FOBYVUs2z06ssQ3,https://api.spotify.com/v1/tracks/1bB1538FOBYV...,https://api.spotify.com/v1/audio-analysis/1bB1...,258873,4,You Never Say You're Sorry,Martin Landström
1,0.510,0.189,6,-15.775,1,0.0341,0.972,0.928,0.1000,0.2160,103.792,audio_features,2nUhKiUfVdLcGmcdYHdjBw,spotify:track:2nUhKiUfVdLcGmcdYHdjBw,https://api.spotify.com/v1/tracks/2nUhKiUfVdLc...,https://api.spotify.com/v1/audio-analysis/2nUh...,169863,4,Last Call,Glass Trio
2,0.634,0.232,3,-18.569,1,0.0373,0.970,0.902,0.1050,0.3310,122.381,audio_features,1H460eor4CG8p8OAz7TGYM,spotify:track:1H460eor4CG8p8OAz7TGYM,https://api.spotify.com/v1/tracks/1H460eor4CG8...,https://api.spotify.com/v1/audio-analysis/1H46...,220410,4,Love Walked In,Gene Hart Trio
3,0.555,0.240,0,-15.647,1,0.0348,0.977,0.839,0.1380,0.3940,127.822,audio_features,2xVVCXzIuILeIRoQrZYlQD,spotify:track:2xVVCXzIuILeIRoQrZYlQD,https://api.spotify.com/v1/tracks/2xVVCXzIuILe...,https://api.spotify.com/v1/audio-analysis/2xVV...,208096,4,My Friend In Tanganyika,Nelly Vernon
4,0.433,0.267,5,-15.967,1,0.0388,0.973,0.896,0.1030,0.0928,119.322,audio_features,2mKGI5TnGVSWbloCZlma6B,spotify:track:2mKGI5TnGVSWbloCZlma6B,https://api.spotify.com/v1/tracks/2mKGI5TnGVSW...,https://api.spotify.com/v1/audio-analysis/2mKG...,191843,4,Giving thrills,Jasmine Patterson


In [26]:
sad_songs = df_audio_features("37i9dQZF1DX7qK8ma5wgG1")
df_list.append(sad_songs)
sad_songs.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.508,0.430,2,-9.475,0,0.0335,0.830,0.000441,0.0930,0.324,130.090,audio_features,1odExI7RdWc4BT515LTAwj,spotify:track:1odExI7RdWc4BT515LTAwj,https://api.spotify.com/v1/tracks/1odExI7RdWc4...,https://api.spotify.com/v1/audio-analysis/1odE...,212954,4,Daylight,David Kushner
1,0.445,0.232,1,-9.145,1,0.0294,0.886,0.000003,0.1280,0.246,72.005,audio_features,0V9QY6NnXtAMwjltgMP0pl,spotify:track:0V9QY6NnXtAMwjltgMP0pl,https://api.spotify.com/v1/tracks/0V9QY6NnXtAM...,https://api.spotify.com/v1/audio-analysis/0V9Q...,204503,4,Face in the Crowd,Freya Ridings
2,0.676,0.461,3,-3.590,1,0.0304,0.555,0.000000,0.0951,0.466,107.950,audio_features,3gdPwk2wyOXNRnTA1KXnEr,spotify:track:3gdPwk2wyOXNRnTA1KXnEr,https://api.spotify.com/v1/tracks/3gdPwk2wyOXN...,https://api.spotify.com/v1/audio-analysis/3gdP...,210880,4,Wish You The Best,Lewis Capaldi
3,0.554,0.326,9,-7.085,1,0.0299,0.697,0.000000,0.0876,0.278,71.267,audio_features,7djOJmSvxXtq0Q49iC9qsX,spotify:track:7djOJmSvxXtq0Q49iC9qsX,https://api.spotify.com/v1/tracks/7djOJmSvxXtq...,https://api.spotify.com/v1/audio-analysis/7djO...,227925,4,Last Train To London,Mimi Webb
4,0.637,0.513,1,-6.797,1,0.0367,0.518,0.000000,0.1240,0.234,141.907,audio_features,22UYp7aAy1XmIYZ1GsnJuB,spotify:track:22UYp7aAy1XmIYZ1GsnJuB,https://api.spotify.com/v1/tracks/22UYp7aAy1Xm...,https://api.spotify.com/v1/audio-analysis/22UY...,202193,4,Part Of Me,Cian Ducrot


In [37]:
lofi_cafe = df_audio_features("37i9dQZF1DX9RwfGbeGQwP")
df_list.append(lofi_cafe)
lofi_cafe.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.746,0.249,9,-16.632,1,0.0539,0.8820,0.925,0.112,0.261,120.043,audio_features,6zkW8DxvN281ruHccv0MIS,spotify:track:6zkW8DxvN281ruHccv0MIS,https://api.spotify.com/v1/tracks/6zkW8DxvN281...,https://api.spotify.com/v1/audio-analysis/6zkW...,129868,4,Pillowy,Cassette Treasures
1,0.684,0.307,1,-11.707,1,0.0514,0.5360,0.960,0.107,0.766,81.986,audio_features,0YexmBFxPG2pLlSTBmiTvW,spotify:track:0YexmBFxPG2pLlSTBmiTvW,https://api.spotify.com/v1/tracks/0YexmBFxPG2p...,https://api.spotify.com/v1/audio-analysis/0Yex...,123659,4,Rock,"DisHand, tillDawn"
2,0.585,0.357,7,-13.928,0,0.0730,0.8460,0.891,0.114,0.505,161.420,audio_features,7BAfo8WKIrAm3hJDj16Xd6,spotify:track:7BAfo8WKIrAm3hJDj16Xd6,https://api.spotify.com/v1/tracks/7BAfo8WKIrAm...,https://api.spotify.com/v1/audio-analysis/7BAf...,144000,4,Mocha,Grizzly Beatz
3,0.698,0.701,0,-8.358,0,0.0465,0.0736,0.787,0.367,0.602,89.993,audio_features,3dnJ9V91tzRy6QjwyagZtu,spotify:track:3dnJ9V91tzRy6QjwyagZtu,https://api.spotify.com/v1/tracks/3dnJ9V91tzRy...,https://api.spotify.com/v1/audio-analysis/3dnJ...,110885,4,sahara,keseki
4,0.677,0.350,11,-10.977,0,0.0601,0.6500,0.901,0.205,0.623,82.008,audio_features,2uhlAgdxgzwMvJ1YEA5ZaT,spotify:track:2uhlAgdxgzwMvJ1YEA5ZaT,https://api.spotify.com/v1/tracks/2uhlAgdxgzwM...,https://api.spotify.com/v1/audio-analysis/2uhl...,121447,4,washed up,Sleepy Titan


In [28]:
happy_hits = df_audio_features("37i9dQZF1DXdPec7aLTmlC")
df_list.append(happy_hits)
happy_hits.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,audio_features,4uUG5RXrOk84mYEfFvj3cK,spotify:track:4uUG5RXrOk84mYEfFvj3cK,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,https://api.spotify.com/v1/audio-analysis/4uUG...,175238,4,I'm Good (Blue),"David Guetta, Bebe Rexha"
1,0.704,0.797,0,-5.927,1,0.0475,0.08260,0.000745,0.0546,0.825,139.994,audio_features,4h9wh7iOZ0GGn8QVp4RAOB,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB,https://api.spotify.com/v1/tracks/4h9wh7iOZ0GG...,https://api.spotify.com/v1/audio-analysis/4h9w...,148486,4,I Ain't Worried,OneRepublic
2,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4,Blinding Lights,The Weeknd
3,0.671,0.845,11,-4.930,0,0.0480,0.02070,0.000000,0.3290,0.775,110.056,audio_features,1vYXt7VSjH9JIM5oRRo7vA,spotify:track:1vYXt7VSjH9JIM5oRRo7vA,https://api.spotify.com/v1/tracks/1vYXt7VSjH9J...,https://api.spotify.com/v1/audio-analysis/1vYX...,176579,4,Dance The Night (From Barbie The Album),Dua Lipa
4,0.548,0.816,0,-4.209,1,0.0465,0.12200,0.000000,0.3350,0.557,95.390,audio_features,6UelLqGlWMcVH1E5c4H7lY,spotify:track:6UelLqGlWMcVH1E5c4H7lY,https://api.spotify.com/v1/tracks/6UelLqGlWMcV...,https://api.spotify.com/v1/audio-analysis/6Uel...,174000,4,Watermelon Sugar,Harry Styles


In [29]:
classical_romance = df_audio_features("37i9dQZF1DX4s3V2rTswzO")
df_list.append(classical_romance)
classical_romance.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.4430,0.05210,10,-22.964,1,0.0492,0.992,0.963,0.1090,0.1870,106.759,audio_features,06naL4CQq3FfhsutfV4rCd,spotify:track:06naL4CQq3FfhsutfV4rCd,https://api.spotify.com/v1/tracks/06naL4CQq3Ff...,https://api.spotify.com/v1/audio-analysis/06na...,123850,4,Sheep May Safely Graze - Arr. by Alexis Ffrench,Alexis Ffrench
1,0.0921,0.01830,7,-31.858,1,0.0513,0.942,0.940,0.1950,0.0390,166.979,audio_features,3JdD9wqvohYMjK6BmEHtTP,spotify:track:3JdD9wqvohYMjK6BmEHtTP,https://api.spotify.com/v1/tracks/3JdD9wqvohYM...,https://api.spotify.com/v1/audio-analysis/3JdD...,172120,3,Satie / Orch. Ducros: Gymnopédie No. 1,"Erik Satie, Gautier Capuçon, Jerome Ducros, Ad..."
2,0.4290,0.04720,11,-27.076,1,0.0870,0.995,0.945,0.0996,0.1560,73.629,audio_features,0OzWMv5ZuMszZVwYUNL9j6,spotify:track:0OzWMv5ZuMszZVwYUNL9j6,https://api.spotify.com/v1/tracks/0OzWMv5ZuMsz...,https://api.spotify.com/v1/audio-analysis/0OzW...,171664,3,Lakmé: Flower Duet (arr. piano),"Léo Delibes, Music Lab Collective"
3,0.2380,0.00484,8,-31.760,1,0.0393,0.995,0.950,0.1210,0.0394,70.821,audio_features,1NechTDnTtSaciAT6gmvJQ,spotify:track:1NechTDnTtSaciAT6gmvJQ,https://api.spotify.com/v1/tracks/1NechTDnTtSa...,https://api.spotify.com/v1/audio-analysis/1Nec...,129920,3,"Schumann: 12 Gedichte aus Liebesfrühling, Op. ...","Clara Schumann, Anneleen Lenaerts, Dionysis Gr..."
4,0.0791,0.05720,7,-29.864,1,0.0494,0.890,0.871,0.3090,0.0386,174.381,audio_features,5JUZN3APQOGsw7LJZYEjmU,spotify:track:5JUZN3APQOGsw7LJZYEjmU,https://api.spotify.com/v1/tracks/5JUZN3APQOGs...,https://api.spotify.com/v1/audio-analysis/5JUZ...,144267,5,Capriol Suite: 5. Pied-en- l'air,"Peter Warlock, Academy of St. Martin in the Fi..."


### Join the dataframes

In [30]:
audio_features_songs = pd.concat(df_list).reset_index(drop=True)
audio_features_songs.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
0,0.812,0.479,2,-5.678,0,0.3330,0.2130,0.000001,0.0756,0.559,169.922,audio_features,2UW7JaomAMuX9pZrjVpHAU,spotify:track:2UW7JaomAMuX9pZrjVpHAU,https://api.spotify.com/v1/tracks/2UW7JaomAMuX...,https://api.spotify.com/v1/audio-analysis/2UW7...,234353,4,La Bebe - Remix,"Yng Lvcas, Peso Pluma"
1,0.652,0.800,9,-4.019,0,0.0614,0.1430,0.629000,0.1120,0.234,143.978,audio_features,7ro0hRteUMfnOioTFI5TG1,spotify:track:7ro0hRteUMfnOioTFI5TG1,https://api.spotify.com/v1/tracks/7ro0hRteUMfn...,https://api.spotify.com/v1/audio-analysis/7ro0...,231704,4,WHERE SHE GOES,Bad Bunny
2,0.748,0.761,2,-6.621,1,0.1020,0.1220,0.000007,0.0984,0.362,116.041,audio_features,5TMLavqGRKNxgzMBrVpC9R,spotify:track:5TMLavqGRKNxgzMBrVpC9R,https://api.spotify.com/v1/tracks/5TMLavqGRKNx...,https://api.spotify.com/v1/audio-analysis/5TML...,179940,4,Mon Amour - Remix,"zzoilo, Aitana"
3,0.748,0.856,11,-3.517,0,0.0348,0.0824,0.000000,0.1420,0.927,123.031,audio_features,0Be7sopyKMv8Y8npsUkax2,spotify:track:0Be7sopyKMv8Y8npsUkax2,https://api.spotify.com/v1/tracks/0Be7sopyKMv8...,https://api.spotify.com/v1/audio-analysis/0Be7...,189320,4,Tacones Rojos,Sebastian Yatra
4,0.783,0.592,11,-8.332,0,0.0331,0.4350,0.000004,0.3470,0.726,120.018,audio_features,7FbrGaHYVDmfr7KoLIZnQ7,spotify:track:7FbrGaHYVDmfr7KoLIZnQ7,https://api.spotify.com/v1/tracks/7FbrGaHYVDmf...,https://api.spotify.com/v1/audio-analysis/7Fbr...,174253,4,Cupid - Twin Ver.,FIFTY FIFTY


In [31]:
# Check for duplicates

audio_features_songs.duplicated().value_counts()

False    883
True       5
dtype: int64

In [32]:
audio_features_songs[audio_features_songs.duplicated()]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist
737,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,audio_features,4uUG5RXrOk84mYEfFvj3cK,spotify:track:4uUG5RXrOk84mYEfFvj3cK,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,https://api.spotify.com/v1/audio-analysis/4uUG...,175238,4,I'm Good (Blue),"David Guetta, Bebe Rexha"
738,0.704,0.797,0,-5.927,1,0.0475,0.08260,0.000745,0.0546,0.825,139.994,audio_features,4h9wh7iOZ0GGn8QVp4RAOB,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB,https://api.spotify.com/v1/tracks/4h9wh7iOZ0GG...,https://api.spotify.com/v1/audio-analysis/4h9w...,148486,4,I Ain't Worried,OneRepublic
740,0.671,0.845,11,-4.930,0,0.0480,0.02070,0.000000,0.3290,0.775,110.056,audio_features,1vYXt7VSjH9JIM5oRRo7vA,spotify:track:1vYXt7VSjH9JIM5oRRo7vA,https://api.spotify.com/v1/tracks/1vYXt7VSjH9J...,https://api.spotify.com/v1/audio-analysis/1vYX...,176579,4,Dance The Night (From Barbie The Album),Dua Lipa
756,0.912,0.774,0,-4.673,0,0.0455,0.12100,0.012700,0.1010,0.632,119.966,audio_features,10xV5l9nhLvFpR8mqzs0bL,spotify:track:10xV5l9nhLvFpR8mqzs0bL,https://api.spotify.com/v1/tracks/10xV5l9nhLvF...,https://api.spotify.com/v1/audio-analysis/10xV...,155589,4,Numb,"Marshmello, Khalid"
775,0.756,0.697,8,-6.377,1,0.0401,0.18200,0.000000,0.3330,0.956,94.996,audio_features,0QBzMgT7NIeoCYy3sJCof1,spotify:track:0QBzMgT7NIeoCYy3sJCof1,https://api.spotify.com/v1/tracks/0QBzMgT7NIeo...,https://api.spotify.com/v1/audio-analysis/0QBz...,206071,4,Bam Bam (feat. Ed Sheeran),"Camila Cabello, Ed Sheeran"


In [33]:
audio_features_songs = audio_features_songs.drop_duplicates()
audio_features_songs.duplicated().value_counts()

False    883
dtype: int64

In [34]:
# Drop the columns that we don't need

audio_features_songs = audio_features_songs.drop(["type", "uri", "track_href", "analysis_url"], axis=1)

In [35]:
# Reorder columns in order display the song id, title and artist at the beginning

audio_features_songs = audio_features_songs[['id', 'title', 'artist', 'danceability', 'energy', 'key', 
                                             'loudness', 'mode', 'speechiness', 'acousticness', 
                                             'instrumentalness', 'liveness', 'valence', 'tempo', 
                                             'duration_ms', 'time_signature']]
audio_features_songs.head()

,id,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2UW7JaomAMuX9pZrjVpHAU,La Bebe - Remix,"Yng Lvcas, Peso Pluma",0.812,0.479,2,-5.678,0,0.3330,0.2130,0.000001,0.0756,0.559,169.922,234353,4
1,7ro0hRteUMfnOioTFI5TG1,WHERE SHE GOES,Bad Bunny,0.652,0.800,9,-4.019,0,0.0614,0.1430,0.629000,0.1120,0.234,143.978,231704,4
2,5TMLavqGRKNxgzMBrVpC9R,Mon Amour - Remix,"zzoilo, Aitana",0.748,0.761,2,-6.621,1,0.1020,0.1220,0.000007,0.0984,0.362,116.041,179940,4
3,0Be7sopyKMv8Y8npsUkax2,Tacones Rojos,Sebastian Yatra,0.748,0.856,11,-3.517,0,0.0348,0.0824,0.000000,0.1420,0.927,123.031,189320,4
4,7FbrGaHYVDmfr7KoLIZnQ7,Cupid - Twin Ver.,FIFTY FIFTY,0.783,0.592,11,-8.332,0,0.0331,0.4350,0.000004,0.3470,0.726,120.018,174253,4


In [36]:
# Save the csv

audio_features_songs.to_csv("audio_features_songs.csv")

### Conclusion
We obtained a csv with 883 unique songs from different styles (pop, latina, jazz, samba, soul, lofi, country, movie soundtracks...) and moods (happy, sad, romance), with their audio features.